In [79]:
import pandas as pd
import numpy as np
import plotly.express as px
import streamlit as st
import altair as alt
from utils_live import teams_dict, get_inside_cut, fix_names, highlight_rows_team_short,plus_prefix
import secrets

In [100]:
# def fix_names(live):
#     """
#     Takes in live datagolf scoring, cleans names, outputs list of active players this week
#     """
#     names = live['player'].str.split(expand=True)                  
#     names[0] = names[0].str.rstrip(",")
#     names[1] = names[1].str.rstrip(",")
#     names['player'] = names[1] + " " + names[0]

#     names['player'] = np.where(names['player']=='Matt Fitzpatrick', 'Matthew Fitzpatrick', names['player'])
#     names['player'] = np.where(names['player']=='Si Kim', 'Si Woo Kim', names['player'])
#     names['player'] = np.where(names['player']=='Min Lee', 'Min Woo Lee', names['player'])
#     names['player'] = np.where(names['player']=='Byeong An', 'Byeong Hun An', names['player'])
#     names['player'] = np.where(names['player']=='Rooyen Van', 'Erik Van Rooyen', names['player'])
#     names['player'] = np.where(names['player']=='Vince Whaley', 'Vincent Whaley', names['player'])
#     names['player'] = np.where(names['player']=='Kevin Yu', 'kevin Yu', names['player'])
#     names['player'] = np.where(names['player']=='Kyounghoon Lee', 'Kyoung-Hoon Lee', names['player'])
#     return names.player

# teams_dict = {
#         '919':'Philly919',
#         'u_c':'unit_circle',
#         'NT 4':'New Team 4',
#         'NT 8':'Sneads Foot',
#         'txms':'txmoonshine',
#         'MG':'Team Gamble',
#         'grrr':'Putt Pirates',
#         '[AW]':'AlphaWired'
#         }

# matchups = {                                    # enter weekly matchups here
#     'unit_circle':1,
#     'Putt Pirates':2,
#     'AlphaWired':3,
#     'txmoonshine':2,
#     'Sneads Foot':1,
#     'New Team 4':4,
#     'Team Gamble':4,
#     'Philly919':3
# }

# def plus_prefix(a):
#     if a > 0:
#         b = f"+{a}"
#     else:
#         b = a
#     return b

# def remove_T_from_positions(dataframe):
#     """
#     Remove 'T' from positions in a given column of a DataFrame.

#     Parameters:
#         dataframe (pandas.DataFrame): DataFrame containing the data.
#         column_name (str): Name of the column containing positions.

#     Returns:
#         pandas.DataFrame: DataFrame with 'T' removed from positions in the specified column.
#     """
#     dataframe['position'] = dataframe['position'].str.replace('T', '')
#     return dataframe

# def get_inside_cut(live_merged):
#     """
#     Filter DataFrame based on position threshold and group by team.

#     Parameters:
#         dataframe (pandas.DataFrame): DataFrame containing the data.
#         position_threshold (str): Threshold position for filtering.

#     Returns:
#         pandas.DataFrame: DataFrame with filtered data grouped by team.
#     """
#     live_merged = live_merged[live_merged['position'] != "WAITING"]
#     live_merged = live_merged[live_merged['position'] != "CUT"]
#     live_merged = remove_T_from_positions(live_merged)
#     live_merged['position'] = live_merged['position'].dropna().astype('int')
#     inside_cut_df = pd.DataFrame(live_merged[live_merged['position'] < 66].team.value_counts()).reset_index()
#     inside_cut_df.columns = ['team','inside_cut']
    
#     inside_cut_dict = dict(inside_cut_df.values)
#     return inside_cut_dict

# def highlight_rows_team_short(row):
#     value = row.loc['Team']
#     if value == 'u_c':
#         color = '#c28bdd' # Purple
#     elif value == '919':
#         color = '#0ec3d2' # Aqua
#     elif value == '[AW]':
#         color = '#f7a05d' # Orange
#     elif value == 'NT 8':
#         color = '#46d671' # Green
#     elif value == 'NT 4':
#         color = '#f75e38' # Red
#     elif value == 'MG':
#         color = '#2693be' # Navy
#     elif value == 'txms':
#         color = '#dbc530' # Yellow
#     else:
#         color = '#73706a' # Grey
#     return ['background-color: {}'.format(color) for r in row]

In [81]:
# get live

dg_key = 'e297e933c3ad47d71ec1626c299e'

path = f"https://feeds.datagolf.com/preds/live-tournament-stats?stats=sg_putt,sg_arg,sg_app,sg_ott,sg_t2g,sg_bs,sg_total,distance,accuracy,gir,prox_fw,prox_rgh,scrambling&round=event_avg&display=value&file_format=csv&key={dg_key}"

live = round(pd.read_csv(path),2).rename(columns={'player_name':'player'})
live = live.set_index(fix_names(live))
live.shape


(89, 23)

In [82]:
# get fantrax teams

teams = pd.read_csv(r"C:\Users\mikej\Desktop\fantrax\week.csv",usecols=['Player','Status','Roster Status'])

teams.columns = ['player','team','active_reserve']
teams['team_short'] = teams['team']
teams['team'] = teams.team.map(teams_dict)
teams = teams.loc[teams.active_reserve=='Active'].set_index('player')
teams.shape

(48, 3)

In [101]:
## MERGE & PROCESS ##

# merge current fantasy teams and live scoring
live_merged = pd.merge(teams, live, how='left', left_index=True, right_index=True)[['team','team_short','position','total','round','thru','sg_putt','sg_arg','sg_app','sg_ott','sg_t2g']].fillna(0).sort_values('total')
live_merged = live_merged.convert_dtypes().reset_index()


# add columns matchup_num & holes_remaining
live_merged['matchup_num'] = live_merged.team.map(matchups)
live_merged['holes_remaining'] = (72 - (live_merged['thru']).fillna(0)).astype(int)
live_merged.loc[live_merged['position'].isin(['CUT', 'WD']), 'holes_remaining'] = 0
print(live_merged.shape)
# live_merged.head()

(48, 12)


In [103]:
# get team leaderboard

team_leaderboard = (live_merged[['team', 'team_short', 'total', 'holes_remaining', 'matchup_num']]
                    .groupby(['team', 'team_short'])
                    .agg({'total': 'sum', 'holes_remaining': 'sum'})
                    .sort_values('total')
                    .reset_index()
                   )

team_leaderboard['inside_cut'] = team_leaderboard['team'].map(get_inside_cut(live_merged)).fillna(0).astype(int)# team_leaderboard['total'] = team_leaderboard['total'].apply(plus_prefix)
team_leaderboard['total'] = team_leaderboard['total'].apply(plus_prefix).astype(int)
team_leaderboard_bar_df = team_leaderboard.copy()

team_leaderboard.drop(columns='team',inplace=True)
team_leaderboard.rename(columns={'team_short':'team'},inplace=True)
team_leaderboard.columns = ['Team','Total','PHR','Inside Cut']
team_leaderboard = team_leaderboard.T.style.apply(highlight_rows_team_short,axis=0)